In [60]:
from sympy.logic.boolalg import to_anf,to_cnf,to_dnf,to_nnf,simplify_logic
from collections import defaultdict
import re
dct = defaultdict(set)
TAUTOLOGY_RE = re.compile(r'~?([a-zA-Z]{1}) ?[|&] ?~?\1')


def _set_identities(fmt_str, name_bool_pairs:list):
    args = []
    
    
    for name,boolean in name_bool_pairs:
        args.append(name if boolean else f'~{name}')
    expr = fmt_str.format(*args)
    anf = to_anf(expr)
    cnf = to_cnf(expr)
    dnf = to_dnf(expr)
    nnf = to_nnf(expr)
    simple = str(simplify_logic(expr))
    bads = [
        # TODO: dynamic
            str('p | ~p'),
            str('p & ~p'),
            str('q | ~q'),
            str('q & ~q'),
#             simple
            ]
    if len(simple) > 2:
        print(f'[{expr}] ≡ {simple}')
        
        if not '^' in (expr_str:=str(expr)) and TAUTOLOGY_RE.search(expr_str) and not expr_str in [simple]:
            dct[simple].add(expr)
        if not '^' in (anf_str:=str(anf)) and not TAUTOLOGY_RE.search(anf_str) and not anf_str in [simple, expr_str]:
            print(f'\tanf: "{anf}"')
            dct[simple].add(anf)
        if not '^' in (cnf_str:=str(cnf)) and not TAUTOLOGY_RE.search(cnf_str) and not cnf_str in [simple, expr_str]:
            print(f'\tcnf: "{cnf}"')
            dct[simple].add(cnf)
        if not '^' in (dnf_str:=str(dnf)) and not TAUTOLOGY_RE.search(dnf_str) and not dnf_str in [simple, expr_str]:
            print(f'\tdnf: "{dnf}"')
            dct[simple].add(dnf)
        if not '^' in (nnf_str:=str(nnf)) and not TAUTOLOGY_RE.search(nnf_str) and not nnf_str in [simple, expr_str]:
            print(f'\tnnf: "{nnf}"')
            dct[simple].add(nnf)
        print()

def set_identities(fmt_str, *symbols):
    for i in (True, False):
        for j in (True, False):
            for k in (True, False):
                pairs = zip(symbols, (i,j,k))
                _set_identities(fmt_str, pairs)
                
                



In [37]:
p_or_q__and_p = set_identities('({} | {}) & {}', 'p','q','p')
p_and_q__or_p = set_identities('({} & {}) | {}', 'p','q','p')
p_or_q__and_q = set_identities('({} | {}) & {}','p','q','q')
p_and_q__or_q = set_identities('({} & {}) | {}','p','q','q')

[(p | q) & ~p] ≡ q & ~p
	anf: "q ^ (p & q)"
	cnf: "~p & (p | q)"
	nnf: "(p | q) & ~p"

[(p | ~q) & ~p] ≡ ~p & ~q
	anf: "p ^ q ^ True ^ (p & q)"
	cnf: "~p & (p | ~q)"
	nnf: "(p | ~q) & ~p"

[(~p | q) & p] ≡ p & q
	cnf: "p & (q | ~p)"
	nnf: "(~p | q) & p"

[(~p | ~q) & p] ≡ p & ~q
	anf: "p ^ (p & q)"
	cnf: "p & (~p | ~q)"
	nnf: "(~p | ~q) & p"

[(p & q) | ~p] ≡ q | ~p
	anf: "p ^ True ^ (p & q)"
	dnf: "~p | (p & q)"
	nnf: "(p & q) | ~p"

[(p & ~q) | ~p] ≡ ~p | ~q
	anf: "True ^ (p & q)"
	dnf: "~p | (p & ~q)"
	nnf: "(p & ~q) | ~p"

[(~p & q) | p] ≡ p | q
	anf: "p ^ q ^ (p & q)"
	dnf: "p | (q & ~p)"
	nnf: "(~p & q) | p"

[(~p & ~q) | p] ≡ p | ~q
	anf: "q ^ True ^ (p & q)"
	dnf: "p | (~p & ~q)"
	nnf: "(~p & ~q) | p"

[(p | q) & ~q] ≡ p & ~q
	anf: "p ^ (p & q)"
	cnf: "~q & (p | q)"
	nnf: "(p | q) & ~q"

[(p | ~q) & q] ≡ p & q
	cnf: "q & (p | ~q)"
	nnf: "(p | ~q) & q"

[(~p | q) & ~q] ≡ ~p & ~q
	anf: "p ^ q ^ True ^ (p & q)"
	cnf: "~q & (q | ~p)"
	nnf: "(~p | q) & ~q"

[(~p | ~q) & q] ≡ q & ~p


In [61]:
set_identities('{} & ({} | {})', 'p','q','r')

[p & (q | r)] ≡ p & (q | r)
	dnf: "(p & q) | (p & r)"

[p & (q | ~r)] ≡ p & (q | ~r)
	dnf: "(p & q) | (p & ~r)"

[p & (~q | r)] ≡ p & (r | ~q)
	dnf: "(p & r) | (p & ~q)"

[p & (~q | ~r)] ≡ p & (~q | ~r)
	dnf: "(p & ~q) | (p & ~r)"

[~p & (q | r)] ≡ ~p & (q | r)
	dnf: "(q & ~p) | (r & ~p)"

[~p & (q | ~r)] ≡ ~p & (q | ~r)
	dnf: "(q & ~p) | (~p & ~r)"

[~p & (~q | r)] ≡ ~p & (r | ~q)
	dnf: "(r & ~p) | (~p & ~q)"

[~p & (~q | ~r)] ≡ ~p & (~q | ~r)
	dnf: "(~p & ~q) | (~p & ~r)"

